### Grupo 02: 
   #### Maryny Zara Castada Collado
   #### Seilys Iglesias Rojas


# Práctica 1. Resolución de problemas con búsqueda 
## Sistemas Inteligentes 2021/2022

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan 2 problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros
 
El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

In [2]:
from search import Problem

In [2]:
# Para que funcione el import el archivo search.py tiene que estar en la mimsa carpeta que este notebook.
# Se puede copiar el archivo search.py a la mimsa carpeta que este notebook.
# Otra opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima/')
import search
import math

In [3]:
## Copiamos aquí la definición de la clase Problem para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [5]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [139]:
p = Jarras()
p.initial

(0, 0)

In [140]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [141]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [142]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [143]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [6]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)
      

In [7]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [146]:
misioneros.initial

(3, 3, 0)

In [147]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [8]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    
    def actions(self,estado):
        pos_hueco = estado.index(0) # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        return accs     

    
    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        
       ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco izquierda":
            if pos_hueco not in (0,3,6):                
                l[pos_hueco] = l[pos_hueco-1]
                l[pos_hueco-1] = 0
        elif accion == "Mover hueco derecha":
            if pos_hueco not in (2,5,8):                
                l[pos_hueco] = l[pos_hueco+1]
                l[pos_hueco+1] = 0    
        
        return tuple(l)
    
    
    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
    
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0

    
    def linear(node):
        #goal = node.state.goal
        goal = (1,2,3,4,5,6,7,8,0)
        return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])


    def manhattan(node):
        state = node.state
        index_goal = {0: [2, 2], 1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1]}
        index_state = {}
        index = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]

        for i in range(len(state)):
            index_state[state[i]] = index[i]

        mhd = 0
        for i in range(len(state)):
            pos_state = index_state[i]
            pos_goal = index_goal[i]

            state_row, state_col = pos_state[0], pos_state[1]
            goal_row, goal_col = pos_goal[0], pos_goal[1]

            mhd += (abs(state_row - goal_row) + abs(state_col - goal_col))

        return mhd
        
    
    def sqrt_manhattan(node):        
        state = node.state
        index_goal = {0: [2, 2], 1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1]}
        index_state = {}
        index = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]

        for i in range(len(state)):
            index_state[state[i]] = index[i]

        mhd = 0
        for i in range(len(state)):
            pos_state = index_state[i]
            pos_goal = index_goal[i]

            state_row, state_col = pos_state[0], pos_state[1]
            goal_row, goal_col = pos_goal[0], pos_goal[1]

            mhd += (abs(state_row - goal_row) + abs(state_col - goal_col))**2

        return math.sqrt(mhd)


    def max_heuristic(node):
        score1 = Ocho_Puzzle.manhattan(node)
        score2 = Ocho_Puzzle.linear(node)
        return max(score1, score2)    

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [149]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [150]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [151]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [152]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [153]:
p8.result(p8.initial,"Mover hueco izquierda")

(2, 8, 3, 1, 6, 4, 0, 7, 5)

In [154]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [4]:
class Quince_Puzzle(Problem): 
    ## Completa la definición del problema
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")        
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
        
        return accs     

    
    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        elif accion == "Mover hueco izquierda":
            if pos_hueco not in (0,4,8,12):               
                l[pos_hueco] = l[pos_hueco-1]
                l[pos_hueco-1] = 0
        elif accion == "Mover hueco derecha":
            if pos_hueco not in (3,7,11,15):                
                l[pos_hueco] = l[pos_hueco+1]
                l[pos_hueco+1] = 0    
        
        return tuple(l)
    
    
    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
       
        
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0
    
    
    def linear(node):
        #goal = node.state.goal
        goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
        return sum([1 if node.state[i] != goal[i] else 0 for i in range(15)])

    
    def manhattan(node):
        state = node.state
        index_goal = {0: [3, 3], 1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [0, 3], 5: [1, 0], 6: [1, 1], 7: [1, 2], 8: [1, 3], 9: [2, 0], 10: [2, 1], 11: [2, 2], 12: [2, 3], 13: [3, 0], 14: [3, 1], 15: [3, 2]}
        index_state = {}
        index = [[0, 0], [0, 1], [0, 2], [0, 3], [1, 0], [1, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]

        for i in range(len(state)):
            index_state[state[i]] = index[i]

        mhd = 0
        for i in range(len(state)):
            pos_state = index_state[i]
            pos_goal = index_goal[i]

            state_row, state_col = pos_state[0], pos_state[1]
            goal_row, goal_col = pos_goal[0], pos_goal[1]

            mhd += (abs(state_row - goal_row) + abs(state_col - goal_col))

        return mhd
        
    
    def sqrt_manhattan(node):        
        state = node.state
        index_goal = {0: [3, 3], 1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [0, 3], 5: [1, 0], 6: [1, 1], 7: [1, 2], 8: [1, 3], 9: [2, 0], 10: [2, 1], 11: [2, 2], 12: [2, 3], 13: [3, 0], 14: [3, 1], 15: [3, 2]}
        index_state = {}
        index = [[0, 0], [0, 1], [0, 2], [0, 3], [1, 0], [1, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 2], [2, 3], [3, 0], [3, 1], [3, 2], [3, 3]]

        for i in range(len(state)):
            index_state[state[i]] = index[i]

        mhd = 0
        for i in range(8):
            pos_state = index_state[i]
            pos_goal = index_goal[i]

            state_row, state_col = pos_state[0], pos_state[1]
            goal_row, goal_col = pos_goal[0], pos_goal[1]

            mhd += (abs(state_row - goal_row) + abs(state_col - goal_col))**2

        return math.sqrt(mhd)
    
    
    def max_heuristic(node):
        score1 = Quince_Puzzle.manhattan(node)
        score2 = Quince_Puzzle.linear(node)
        return max(score1, score2)

Realiza algunas pruebas para comprobar que la clase está bien definida. 

In [156]:
p15 = Quince_Puzzle((7, 13, 12, 3, 0, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6))
p15.initial

(7, 13, 12, 3, 0, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

In [157]:
p15.actions(p15.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco derecha'] 

['Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco derecha']

In [158]:
p15.result(p15.initial,"Mover hueco arriba")
# Respuesta: (0, 13, 12, 3, 7, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

(0, 13, 12, 3, 7, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

In [159]:
p15.result(p15.initial,"Mover hueco abajo")
# Respuesta: (7, 13, 12, 3, 9, 1, 4, 11, 0, 15, 8, 14, 5, 10, 2, 6)

(7, 13, 12, 3, 9, 1, 4, 11, 0, 15, 8, 14, 5, 10, 2, 6)

In [160]:
p15.result(p15.initial,"Mover hueco izquierda")
# Tiene que quedarse igual: (7, 13, 12, 3, 0, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

(7, 13, 12, 3, 0, 1, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

In [161]:
p15.result(p15.initial,"Mover hueco derecha")
# Respuesta: (7, 13, 12, 3, 1, 0, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

(7, 13, 12, 3, 1, 0, 4, 11, 9, 15, 8, 14, 5, 10, 2, 6)

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [5]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [6]:
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima-python/')
import search

In [ ]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [164]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [165]:
breadth_first_graph_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [166]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

#### Misioneros

In [167]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [168]:
breadth_first_graph_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [169]:
depth_first_graph_search(misioneros).solution() 

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

#### Puzzle de 8

In [170]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [171]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [9]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [173]:
sol

'Este estado inicial no tiene solucion'

In [174]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [175]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [176]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [177]:
# Tiene solucion pero tarda en encontrarla
depth_first_graph_search(estado).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover h

In [179]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### Puzzle de 16

In [7]:
# Comprueba si el estado inicial (0, 1, 3, 4, 2, 6, 7, 8, 5, 10, 11, 12, 9, 13, 14, 15) tiene solución.

p15 = Quince_Puzzle((0, 1, 3, 4, 2, 6, 7, 8, 5, 10, 11, 12, 9, 13, 14, 15))

if p15.check_solvability(p15.initial):
    sol = "Este estado inicial tiene solucion"
else: 
    sol = "Este estado inicial no tiene solucion"

In [8]:
sol

'Este estado inicial tiene solucion'

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- **Misioneros**: Para el caso de busqueda en anchura, tanto con tree search y graph search se obtiene una solución inmediata para la resolución de este problema. Donde se queda en [*] es para el caso de búsqueda en profundidad con tree search, porque al no realizar control de repetidos se queda en un bucle infinito, a diferencia de si se usa graph search, con el cual también se obtiene una solución inmediata al haber control de repetidos.
- **Puzle de 8**: Si se usa tree search, ya sea con el algoritmo de anchura como de profundidad, dependiendo del estado inicial que se pasa al problema, es posible que se quede en un bucle infinito. En los casos donde se usa graph search, tanto con el algoritmo de anchura como en profundidad, se obtiene una solución, pero que tarda en encontrarse.
- **Puzle de 16**: Para el caso de usar tree search ocurre lo mismo que en el puzzle de 8. Donde se diferencia es en el uso de graph search, donde al ser un problema con una dimensión mayor que el de puzle de 8, en el caso de usar el algoritmo de profundidad, como no es un algoritmo completo, dependiendo del estado inicial que se pase, puede meterse en caminos infinitos y no llegar a encontrar una solución. 

**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [ ]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
# Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.


### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

#### 8-puzzle

In [12]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [49]:
astar_search(puzle,Ocho_Puzzle.manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [50]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [51]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [52]:
astar_search(puzle,Ocho_Puzzle.linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [53]:
astar_search(puzle,Ocho_Puzzle.max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### 16-puzzle

In [11]:
####  Vamos a probar el 16-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+---+
#              | 1 | 2 | 3 | 4 |
#              +---+---+---+---+
#              | 5 | 6 | 7 | 8 |
#              +---+---+---+---+
#              | 9 | 12 |10 |15 |
#              +---+---+---+---+
#              |13 | H |11 |14 |
#              +---+---+---+---+

qpuzle = Quince_Puzzle((1,2,3,4,5,6,7,8,9,12,10,15,13,0,11,14)) 

In [85]:
astar_search(qpuzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [86]:
astar_search(qpuzle,Quince_Puzzle.manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [87]:
astar_search(qpuzle,Quince_Puzzle.linear).solution()
# En este caso, a diferencia de la ejecución anterior donde solo se usa A*, no tarda mucho en devolver una solucion

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [88]:
astar_search(qpuzle,Quince_Puzzle.max_heuristic).solution()
# En este caso, a diferencia de la ejecución anterior donde solo se usa A*, no tarda mucho en devolver una solucion

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [19]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [60]:
%%timeit
astar_search(puzzle_1, Ocho_Puzzle.linear)
astar_search(puzzle_2, Ocho_Puzzle.linear)
astar_search(puzzle_3, Ocho_Puzzle.linear)

3.09 ms ± 416 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
%%timeit
astar_search(puzzle_1, Ocho_Puzzle.manhattan)
astar_search(puzzle_2, Ocho_Puzzle.manhattan)
astar_search(puzzle_3, Ocho_Puzzle.manhattan)

2.51 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
astar_search(puzzle_1, Ocho_Puzzle.sqrt_manhattan)
astar_search(puzzle_2, Ocho_Puzzle.sqrt_manhattan)
astar_search(puzzle_3, Ocho_Puzzle.sqrt_manhattan)

17.1 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
%%timeit
astar_search(puzzle_1, Ocho_Puzzle.max_heuristic)
astar_search(puzzle_2, Ocho_Puzzle.max_heuristic)
astar_search(puzzle_3, Ocho_Puzzle.max_heuristic)

2.52 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
astar_search(puzzle_1, Ocho_Puzzle.linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
astar_search(puzzle_2, Ocho_Puzzle.manhattan).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [59]:
astar_search(puzzle_3, Ocho_Puzzle.sqrt_manhattan).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha']

In [60]:
astar_search(puzzle_3, Ocho_Puzzle.max_heuristic).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha']

In [14]:
qpuzzle = Quince_Puzzle((1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0))

In [186]:
%%timeit
astar_search(qpuzzle, Quince_Puzzle.linear)

141 µs ± 6.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
astar_search(qpuzzle, Quince_Puzzle.linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [187]:
%%timeit
astar_search(qpuzzle, Quince_Puzzle.manhattan)

429 µs ± 130 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [45]:
astar_search(qpuzzle, Quince_Puzzle.manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [17]:
%%timeit
astar_search(qpuzzle, Quince_Puzzle.sqrt_manhattan)

388 µs ± 59.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
astar_search(qpuzzle, Quince_Puzzle.sqrt_manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [189]:
%%timeit
astar_search(qpuzzle, Quince_Puzzle.max_heuristic)

627 µs ± 214 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [48]:
astar_search(qpuzzle, Quince_Puzzle.max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.



## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [9]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [78]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [95]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [96]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [97]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, Ocho_Puzzle.manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [98]:
astar_search(p8, Ocho_Puzzle.manhattan).solution()

['Mover hueco derecha']

In [99]:
astar_search(p8p, Ocho_Puzzle.manhattan).solution()

['Mover hueco derecha']

In [13]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [86]:
resuelve_ocho_puzzle(estado_inicial,astar_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


In [14]:
def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):
    p15=Quince_Puzzle(estado_inicial)
    p15p=Problema_con_Analizados(p15)
    if p15.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p15p,h).solution()
        else: 
            sol= algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p15p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [15]:
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)

#### Anchura

In [34]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,breadth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 19773
14.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [35]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,breadth_first_graph_search)

Este problema no tiene solucion. 
116 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [36]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,breadth_first_graph_search)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 57057
1min 36s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [37]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 13
433 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [38]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 28. Nodos analizados: 178224
12min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### Profundidad

In [39]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco der

In [43]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,depth_first_graph_search)

Este problema no tiene solucion. 
269 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquie

In [45]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: depth_first_graph_search
Longitud de la solución: 27. Nodos analizados: 28
1.18 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [46]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arri

#### Coste uniforme

In [47]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14105
53.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [48]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,uniform_cost_search)

Este problema no tiene solucion. 
418 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [49]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428
12min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [50]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 3. Nodos analizados: 13
1.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [52]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 28. Nodos analizados: 177480
1h 45min 29s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### Primero el mejor (linear) 

In [53]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,best_first_graph_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritm

In [54]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,best_first_graph_search,Ocho_Puzzle.linear)

Este problema no tiene solucion. 
231 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [55]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,best_first_graph_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [56]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,best_first_graph_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
327 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [57]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,best_first_graph_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 29
12 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### Primero el mejor (manhattan) 

In [58]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,best_first_graph_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover h

In [59]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,best_first_graph_search,Ocho_Puzzle.manhattan)

Este problema no tiene solucion. 
185 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [60]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,best_first_graph_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco 

In [88]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,best_first_graph_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
533 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [62]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,best_first_graph_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 28. Nodos analizados: 29
1.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### A* (linear)

In [63]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,astar_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 640
114 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [64]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,astar_search,Ocho_Puzzle.linear)

Este problema no tiene solucion. 
128 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [65]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,astar_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3230
3.88 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [66]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,astar_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
381 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [67]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,astar_search,Ocho_Puzzle.linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 53474
20min 20s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### A* (manhattan)

In [68]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E1,astar_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 183
25.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [69]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E2,astar_search,Ocho_Puzzle.manhattan)

Este problema no tiene solucion. 
139 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E3,astar_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 1153
372 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [71]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E4,astar_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
751 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [72]:
%timeit -r 1 -n 1 resuelve_ocho_puzzle(E5,astar_search,Ocho_Puzzle.manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 28. Nodos analizados: 805
208 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:


          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 2 | 7 | 3 |   | 3 | 0 | 1 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |    | H | 8 | 7 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 4 | 6 | 8 |   | 8 | 4 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |    | 6 | 5 | 4 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 1 | H | 5 |   | 5 | 2 | 7 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |    | 3 | 2 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                    E1              E2          E3          E4           E5
    Anchura                      L=17       L=Sin solucion    L=20        L=3        L=28
                                 T=14.4 s   T=116 µs          T=1min 36s  T=433 µs   T=12min 16s
                                 NA=19773   NA=Sin solucion   NA=57057    NA=13      NA=178224
   
    Profundidad                  L=55689    L=Sin solucion    L=39720     L=27        L=64830
                                 T=9min 4s  T=269 µs          T=4min 19s  T=1.18 ms   T=18min 50s
                                 NA=65442   NA=Sin solucion   NA=42926    NA=28       NA=106784
                                       
    Coste Uniforme               L=17      L=Sin solucion    L=20        L=3         L=28
                                 T=53.9 s  T=418 µs          T=12min 15s T=1.5 ms    T=1h 45min 29s
                                 NA=14105  NA=Sin solucion   NA=48428    NA=13       NA=177480
                                       
    Primero el mejor (linear)    L=43      L=Sin solucion    L=76        L=3         L=28
                                 T=213 ms  T=231 µs          T=189 ms    T=327 µs    T=12 ms 
                                 NA=690    NA=Sin solucion   NA=747      NA=4        NA=29 
                                                                            
    Primero el mejor (manhattan) L=93      L=Sin solucion    L=74        L=3         L=28
                                 T=227 ms  T=185 µs          T=153 ms    T=533 µs    T=1.9 ms
                                 NA=699    NA=Sin solucion   NA=591      NA=4        NA=29
                                                                             
    A* (linear)                  L=17      L=Sin solucion    L=20        L=3         L=28
                                 T=114 ms  T=128 µs          T=3.88 s    T=381 µs    T=20min 20s
                                 NA=640    NA=Sin solucion   NA=3230     NA=4        NA=53474
                                                                             
    A* (manhattan)               L=17       L=Sin solucion   L=20        L=3         L=28 
                                 T=25.5 ms  T=139 µs         T=372 ms    T=751 µs    T=208 ms
                                 NA=183     NA=Sin solucion  NA=1153     NA=4        NA=805


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 

**Escribe aquí tus conclusiones justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
 Con los resultados obtenidos se puede observar que usando los algoritmos búsqueda en anchura, coste uniforme y A* (con el uso de las heurísticas linear como manhattan), se obtiene una solución con una longitud menor comparado con el uso de los algoritmos de búsqueda en profundidad y primero el mejor. 
 En el caso de los algoritmos donde hemos obtenido una longitud de la solución menor (búsqueda en anchura, coste uniforme y A*), el mejor resultado que se obtiene es con el uso de A*, dado que los nodos analizados que se obtienen para obtener el mejor camino es menor y, como en A* el objetivo es obtener el mejor camino de coste minimo, se puede observar que se invierte una gran cantidad de tiempo para dicha búsqueda.



### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales:

In [12]:
Q1 = (2,5,12,1,13,0,8,10,6,14,9,7,15,3,11,4)
# Q2 = (8,3,15,14,1,0,10,12,13,5,7,2,4,6,11,9)
# Q3 = (13,6,14,3,5,1,9,11,4,8,7,0,2,15,12,10)
# Q4 = (3,2,11,10,12,13,8,6,1,9,7,0,15,4,14,5)
# Q5 = (3,0,13,11,7,8,4,10,14,9,1,12,2,5,15,6)

#### Anchura

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,breadth_first_graph_search)

In [ ]:
# %timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,breadth_first_graph_search)

In [ ]:
# %timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,breadth_first_graph_search)

In [ ]:
# %timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,breadth_first_graph_search)

In [ ]:
# %timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,breadth_first_graph_search)

#### Profundidad

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,depth_first_graph_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,depth_first_graph_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,depth_first_graph_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,depth_first_graph_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,depth_first_graph_search)

#### Coste uniforme

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,uniform_cost_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,uniform_cost_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,uniform_cost_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,uniform_cost_search)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,uniform_cost_search)

#### Primero el mejor (linear) 

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,best_first_graph_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,best_first_graph_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,best_first_graph_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,best_first_graph_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,best_first_graph_search,Quince_Puzzle.linear)

#### Primero el mejor (manhattan) 

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,best_first_graph_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,best_first_graph_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,best_first_graph_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,best_first_graph_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,best_first_graph_search,Quince_Puzzle.manhattan)

#### A* (linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,astar_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,astar_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,astar_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,astar_search,Quince_Puzzle.linear)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,astar_search,Quince_Puzzle.linear)

#### A* (manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q1,astar_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q2,astar_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q3,astar_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q4,astar_search,Quince_Puzzle.manhattan)

In [ ]:
%timeit -r 1 -n 1 resuelve_quince_puzzle(Q5,astar_search,Quince_Puzzle.manhattan)

                                       Q1           Q2           Q3          Q4        Q5
    Anchura                            L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
   
    Profundidad                        L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Coste Uniforme                     L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Primero el mejor (linear)          L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T= 
                                       NA=          NA=          NA=         NA=       NA= 
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L=        L= 
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                                                              
    A* (linear)                        L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                                                             
    A* (manhattan)                     L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=


 -----------------------------------------------------------------------------------------

Escribe aquí tus conclusiones sobre los resultados y la eficiencia de las distintas opciones. 



###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 
